<a href="https://colab.research.google.com/github/shahrazzkhmohammed/learn-python/blob/master/NLP_day2_May19_G1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

List of important packages to be used in NLP:

- NLTK

- BERT or transformers

- Spacy

- Textblob

- string

- ...



What can we do with BERT today?

NER, Text summarization, BQA


NER is Named-entity recognition.

By NER we can determine the type and role of the words in the sentences.

https://en.wikipedia.org/wiki/Named-entity_recognition

Named-entity recognition (NER) (also known as (named) entity identification, entity chunking, and entity extraction) is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.







When we work with BERT, the first step is always tokenization of the text. Within a pipeline of tasks, there might be another step to do (other than tokenization) and it can be for example token classification.



In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 13.0MB/s 
     |████████████████████████████████| 3.3MB 49.6MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 


In [2]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Allocate a pipeline for sentiment-analysis
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
nlp = pipeline('ner', model=model, tokenizer=tokenizer)


In [6]:
 nlp('Enzo works at the Australian National University (AUN)')

[{'end': 2,
  'entity': 'I-PER',
  'index': 1,
  'score': 0.9983270168304443,
  'start': 0,
  'word': 'En'},
 {'end': 4,
  'entity': 'I-PER',
  'index': 2,
  'score': 0.9952995777130127,
  'start': 2,
  'word': '##zo'},
 {'end': 28,
  'entity': 'I-ORG',
  'index': 6,
  'score': 0.9984350204467773,
  'start': 18,
  'word': 'Australian'},
 {'end': 37,
  'entity': 'I-ORG',
  'index': 7,
  'score': 0.9967807531356812,
  'start': 29,
  'word': 'National'},
 {'end': 48,
  'entity': 'I-ORG',
  'index': 8,
  'score': 0.9959043264389038,
  'start': 38,
  'word': 'University'},
 {'end': 52,
  'entity': 'I-ORG',
  'index': 10,
  'score': 0.9900023937225342,
  'start': 50,
  'word': 'AU'},
 {'end': 53,
  'entity': 'I-ORG',
  'index': 11,
  'score': 0.976391077041626,
  'start': 52,
  'word': '##N'}]

In [4]:
type(a)

list

In [7]:
nlp('David works at Google.')

[{'end': 5,
  'entity': 'I-PER',
  'index': 1,
  'score': 0.9979717135429382,
  'start': 0,
  'word': 'David'},
 {'end': 21,
  'entity': 'I-ORG',
  'index': 4,
  'score': 0.9990472793579102,
  'start': 15,
  'word': 'Google'}]

In [12]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

sequence = "Rabjot Singh and Vijay are attending the NLP class in Lambton College!"

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

[('[CLS]', 'O'), ('Ra', 'I-PER'), ('##b', 'I-PER'), ('##jo', 'I-PER'), ('##t', 'I-PER'), ('Singh', 'I-PER'), ('and', 'O'), ('Vijay', 'I-PER'), ('are', 'O'), ('attending', 'O'), ('the', 'O'), ('NL', 'I-MISC'), ('##P', 'O'), ('class', 'O'), ('in', 'O'), ('Lamb', 'I-ORG'), ('##ton', 'I-ORG'), ('College', 'I-ORG'), ('!', 'O'), ('[SEP]', 'O')]


There are some problems with NER process with BERT; the names are broken down incorrectly into tokens and then we cannot find non-English names and we cannot find the long names.

In [13]:
text = "Rabjot Singh and Vijay are attending the NLP class in Lambton College!"

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
inputs = tokenizer.encode(text, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

[('[CLS]', 'O'), ('Ra', 'I-PER'), ('##b', 'I-PER'), ('##jo', 'I-PER'), ('##t', 'I-PER'), ('Singh', 'I-PER'), ('and', 'O'), ('Vijay', 'I-PER'), ('are', 'O'), ('attending', 'O'), ('the', 'O'), ('NL', 'I-MISC'), ('##P', 'O'), ('class', 'O'), ('in', 'O'), ('Lamb', 'I-ORG'), ('##ton', 'I-ORG'), ('College', 'I-ORG'), ('!', 'O'), ('[SEP]', 'O')]


We need post-processing to get the right NER output with BERT technology.

The best package to discover the full name of a person within a text is flair.



In [14]:
!pip install flair
# reference to flair: https://github.com/flairNLP/flair

ERROR: konoha 4.6.4 has requirement requests<3.0.0,>=2.25.1, but you'll have requests 2.23.0 which is incompatible.
  Found existing installation: importlib-metadata 4.0.1
    Uninstalling importlib-metadata-4.0.1:
      Successfully uninstalled importlib-metadata-4.0.1


In [1]:
from flair.data import Sentence
from flair.models import SequenceTagger

# make a sentence
sentence = Sentence('I love Berlin .')

# load the NER tagger
tagger = SequenceTagger.load('ner')

# run NER over sentence
tagger.predict(sentence)

2021-05-19 15:17:23,417 --------------------------------------------------------------------------------
2021-05-19 15:17:23,419 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-05-19 15:17:23,423  - The most current version of the model is automatically downloaded from there.
2021-05-19 15:17:23,425  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-05-19 15:17:23,427 --------------------------------------------------------------------------------



2021-05-19 15:17:36,790 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [2]:
print(sentence)
print('The following NER tags are found:')

# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence: "I love Berlin ."   [− Tokens: 4  − Token-Labels: "I love Berlin <S-LOC> ."]
The following NER tags are found:
Span [3]: "Berlin"   [− Labels: LOC (0.999)]


In [5]:
text = "William Henry Gates III is working in his office in Microsoft!"
text = Sentence(text)

tagger.predict(text)


In [6]:
for entity in text.get_spans('ner'):
    print(entity)

Span [1,2,3,4]: "William Henry Gates III"   [− Labels: PER (0.9396)]
Span [11,12]: "Microsoft !"   [− Labels: ORG (0.6496)]


In [7]:
text = "William Henry Gates III is working in his office in Microsoft!"
text = Sentence(text)

tagger.predict(text)

entity_dict = text.to_dict(tag_type="ner")


ListOfNamesInText = []
for e in entity_dict['entities']:
  if str(e["labels"][0]).split()[0] == "PER":
    ListOfNamesInText.append(e["text"])
ListOfNamesInText


['William Henry Gates III']

In [8]:
text = "Rabjot Singh and Vijay are attending the NLP class in Lambton College!"
text = Sentence(text)

tagger.predict(text)

entity_dict = text.to_dict(tag_type="ner")


ListOfNamesInText = []
for e in entity_dict['entities']:
  if str(e["labels"][0]).split()[0] == "PER":
    ListOfNamesInText.append(e["text"])
ListOfNamesInText


['Rabjot Singh', 'Vijay']

In [9]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(summarizer(ARTICLE, max_length=130, min_length=30))


[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . At one time, she was married to eight men at once, prosecutors say .'}]
